In [1]:
import pandas as pd
import numpy as np
import scipy.stats as sts
import matplotlib.pyplot as plt
%matplotlib inline
import statsmodels.stats.multitest as smm

In [8]:
df = pd.read_csv("gene_high_throughput_sequencing.csv")
genes = df.columns[2:]
normal = df[df.Diagnosis == 'normal']
neoplasia = df[df.Diagnosis == 'early neoplasia']
cancer = df[df.Diagnosis == 'cancer']
df.head()


,Patient_id,Diagnosis,LOC643837,LOC100130417,SAMD11,NOC2L,KLHL17,PLEKHN1,C1orf170,HES4,...,CLIC2,RPS4Y1,ZFY,PRKY,USP9Y,DDX3Y,CD24,CYorf15B,KDM5D,EIF1AY
0,STT5425_Breast_001_normal,normal,1.257614,2.408148,13.368622,9.494779,20.880435,12.722017,9.494779,54.349694,...,4.761250,1.257614,1.257614,1.257614,1.257614,1.257614,23.268694,1.257614,1.257614,1.257614
1,STT5427_Breast_023_normal,normal,4.567931,16.602734,42.477752,25.562376,23.221137,11.622386,14.330573,72.445474,...,6.871902,1.815112,1.815112,1.815112,1.815112,1.815112,10.427023,1.815112,1.815112,1.815112
2,STT5430_Breast_002_normal,normal,2.077597,3.978294,12.863214,13.728915,14.543176,14.141907,6.232790,57.011005,...,7.096343,2.077597,2.077597,2.077597,2.077597,2.077597,22.344226,2.077597,2.077597,2.077597
3,STT5439_Breast_003_normal,normal,2.066576,8.520713,14.466035,7.823932,8.520713,2.066576,10.870009,53.292034,...,5.200770,2.066576,2.066576,2.066576,2.066576,2.066576,49.295538,2.066576,2.066576,2.066576
4,STT5441_Breast_004_normal,normal,2.613616,3.434965,12.682222,10.543189,26.688686,12.484822,1.364917,67.140393,...,11.227770,1.364917,1.364917,1.364917,1.364917,1.364917,23.627911,1.364917,1.364917,1.364917


In [11]:
#1 задание
alpha = 0.05
count1 = 0
count2 = 0
for gene in genes:
    z, p1 = sts.ttest_ind(normal[gene], neoplasia[gene], equal_var=False)
    z, p2 = sts.ttest_ind(neoplasia[gene], cancer[gene], equal_var=False)

    if p1 < alpha:
        count1 += 1
        
    if p2 < alpha:
        count2 += 1

print("Число генов, со статистически значимыми различиями между normal и early neoplasia:", count1)
print("Число генов, со статистически значимыми различиями между early neoplasia и cancer:", count2)

Число генов, со статистически значимыми различиями между normal и early neoplasia: 1575
Число генов, со статистически значимыми различиями между early neoplasia и cancer: 3490


In [31]:
#2 задание
alpha = 0.05 / 2
count1 = 0
count2 = 0
p_vals1 = []
p_vals2 = []
for gene in genes:
    z, p1 = sts.ttest_ind(normal[gene], neoplasia[gene], equal_var=False)
    z, p2 = sts.ttest_ind(neoplasia[gene], cancer[gene], equal_var=False)
    p_vals1.append(p1)
    p_vals2.append(p2)

is_sign1, p_vals1_corr, c, d = smm.multipletests(p_vals1, alpha=alpha, method='holm')
is_sign2, p_vals2_corr, c, d = smm.multipletests(p_vals2, alpha=alpha, method='holm')

count1 = np.sum(number_of_significant(is_sign1, normal, neoplasia))
count2 = np.sum(number_of_significant(is_sign2, neoplasia, cancer))

print("Число генов, со статистически значимыми различиями между normal и early neoplasia:", count1)
print("Число генов, со статистически значимыми различиями между early neoplasia и cancer:", count2)

Число генов, со статистически значимыми различиями между normal и early neoplasia: 2
Число генов, со статистически значимыми различиями между early neoplasia и cancer: 77


In [30]:
def number_of_significant(is_sign, control, treatment):

    result = []
    for i, sign in enumerate(is_sign):
        if sign:
            C = control[genes[i]].mean()
            T = treatment[genes[i]].mean()

            if C > T:
                Fc = - C / T
            else:
                Fc = T / C

            if np.abs(Fc) > 1.5:
                result.append(1)
            else:
                result.append(0)
        else:
            result.append(0)

    return result

In [29]:
#3 задание
alpha = 0.05 / 2
count1 = 0
count2 = 0
p_vals1 = []
p_vals2 = []
for gene in genes:
    z, p1 = sts.ttest_ind(normal[gene], neoplasia[gene], equal_var=False)
    z, p2 = sts.ttest_ind(neoplasia[gene], cancer[gene], equal_var=False)
    p_vals1.append(p1)
    p_vals2.append(p2)

is_sign1, p_vals1_corr, c, d = smm.multipletests(p_vals1, alpha=alpha, method="fdr_bh")
is_sign2, p_vals2_corr, c, d = smm.multipletests(p_vals2, alpha=alpha, method="fdr_bh")

count1 = np.sum(number_of_significant(is_sign1, normal, neoplasia))
count2 = np.sum(number_of_significant(is_sign2, neoplasia, cancer))

print("Число генов, со статистически значимыми различиями между normal и early neoplasia:", count1)
print("Число генов, со статистически значимыми различиями между early neoplasia и cancer:", count2)

Число генов, со статистически значимыми различиями между normal и early neoplasia: 4
Число генов, со статистически значимыми различиями между early neoplasia и cancer: 524
